In [57]:
import yfinance as yf
import pandas as pd
import warnings
import plotly.express as px

from optimisation_markowitz import MarkowitzOptimization
from optimisation_simm import SingleFactorModel, SingleFactorModelOptimization
from utils import Utils
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
start_date = "2023-04-01"
end_date = "2024-04-01"
annualized_factor = 252

We will use S&P500 Healthcare Index as benchmark to evaluate the performance with Single Index Market Model and Constant Correlation Model. The return over four years will be used as the market return in the formula

In [59]:
# Get close data for S&P 500 Information Technology Index
market_index_data = Utils.get_historical_data("^SP500-35", start_date, end_date)

# calculate simple return for market index
market_index_simple_return = Utils.calculate_simple_return(market_index_data)

# calculate daily market data returns.
# use that to calculate expected returns and variance
market_index_daily_returns = Utils.calculate_daily_returns(market_index_data)
market_index_data_expected_return = market_index_daily_returns.mean()
market_index_data_variance = market_index_daily_returns.var()

print(f"Market Index Simple Return from {start_date} - {end_date} in %: ", market_index_simple_return * 100)
print("Market Index Expected daily Return in %: ", market_index_data_expected_return * 100)
print("Market Index daily Variance in %: ", market_index_data_variance * 100)

[*********************100%%**********************]  1 of 1 completed
Market Index Simple Return from 2023-04-01 - 2024-04-01 in %:  12.896186517907182
Market Index Expected daily Return in %:  0.051107595660645815
Market Index daily Variance in %:  0.004390164722301008


For markowitz model, we will use 3-Month US Treasury Bill as risk-free rate. Since we are holding the portfolio for 1 year, we will take the last value as the risk-free rate.

In [60]:
risk_free_rate_df = yf.download("^IRX", start_date, end_date)
risk_free_rate = risk_free_rate_df['Adj Close'].iloc[-1] / 100
risk_free_rate

[*********************100%%**********************]  1 of 1 completed


0.05203000068664551

# Healthcare Sector ETFs

We will now download the data for US Healthcare Equity ETFs on which we will run our strategies. We will use the following top 5 ETFs based on total assets:
1. Healthcare Select Sector SPDR Fund (XLV)
2. Vanguard Health Care ETF (VHT)
3. iShares Biotechnology ETF (IBB)
4. SPDR S&P Biotech ETF (XBI)
5. iShares U.S. Medical Devices ETF (IHI)

Source: [https://etfdb.com/etfdb-category/technology-equities/ ](https://etfdb.com/etfdb-category/health-biotech-equities/)

In [61]:
# get the data
xlv_data = Utils.get_historical_data("XLV", start_date, end_date)
vht_data = Utils.get_historical_data("VHT", start_date, end_date)
ibb_data = Utils.get_historical_data("IBB", start_date, end_date)
xbi_data = Utils.get_historical_data("XBI", start_date, end_date)
ihi_data = Utils.get_historical_data("IHI", start_date, end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [62]:
# check for any missing data
print(market_index_data.isnull().sum())
print(xlv_data.isnull().sum())
print(vht_data.isnull().sum())
print(ibb_data.isnull().sum())
print(xbi_data.isnull().sum())
print(ihi_data.isnull().sum())


Adj Close        0
simple_return    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64


In [63]:

xlv_simple_return = Utils.calculate_simple_return(xlv_data)
vht_simple_return = Utils.calculate_simple_return(vht_data)
ibb_simple_return = Utils.calculate_simple_return(ibb_data)
xbi_simple_return = Utils.calculate_simple_return(xbi_data)
ihi_simple_return = Utils.calculate_simple_return(ihi_data)

simple_returns_dict = {
    "XLV": xlv_simple_return,
    "VHT": vht_simple_return,
    "IBB": ibb_simple_return,
    "XBI": xbi_simple_return,
    "IHI": ihi_simple_return
}

for key, value in simple_returns_dict.items():
    print(f"{key} Simple Return from {start_date} - {end_date} in %: ", value * 100)

XLV Simple Return from 2023-04-01 - 2024-04-01 in %:  14.699930196487587
VHT Simple Return from 2023-04-01 - 2024-04-01 in %:  13.941585714239867
IBB Simple Return from 2023-04-01 - 2024-04-01 in %:  6.07511630652855
XBI Simple Return from 2023-04-01 - 2024-04-01 in %:  23.162156584619463
IHI Simple Return from 2023-04-01 - 2024-04-01 in %:  9.752419566907665


In [64]:
# plot the simple retruns using plotly for all stocks along with market index
fig = px.line(title="Simple Returns for HealthCare Stocks and Market Index")
fig.add_scatter(x=xlv_data.index, y=xlv_data['simple_return'], mode='lines', name='XLV')
fig.add_scatter(x=vht_data.index, y=vht_data['simple_return'], mode='lines', name='VHT')
fig.add_scatter(x=ibb_data.index, y=ibb_data['simple_return'], mode='lines', name='IBB')
fig.add_scatter(x=xbi_data.index, y=xbi_data['simple_return'], mode='lines', name='XBI')
fig.add_scatter(x=ihi_data.index, y=ihi_data['simple_return'], mode='lines', name='IHI')
fig.add_scatter(x=market_index_data.index, y=market_index_data['simple_return'], mode='lines', name='Market Index')
fig.show()

On observing the simple returns, we see that SPDR S&P Biotech ETF (XBI) had the best performance, even outperforming the market. This ETF consists of many small-cap biotech companies, which are known for their high volatility providing it as a speculative investment which is justified by the sharper U-Shape returns curve.

In [65]:
xlv_daily_returns = Utils.calculate_daily_returns(xlv_data)
vht_daily_returns = Utils.calculate_daily_returns(vht_data)
ibb_daily_returns = Utils.calculate_daily_returns(ibb_data)
xbi_daily_returns = Utils.calculate_daily_returns(xbi_data)
ihi_daily_returns = Utils.calculate_daily_returns(ihi_data)

daily_returns_df = pd.DataFrame([xlv_daily_returns, vht_daily_returns, ibb_daily_returns, xbi_daily_returns, ihi_daily_returns]).T
daily_returns_df.columns = ['XLV', 'VHT', 'IBB', 'XBI', 'IHI']

In [66]:
# we can now plot the daily returns for all the stocks in sub plots
fig = make_subplots(rows=3, cols=2, subplot_titles=("XLV", "VHT", "IBB", "XBI", "IHI", "Market Index"))

fig.add_scatter(x=xlv_data.index, y=xlv_daily_returns, mode='lines', name='XLV', row=1, col=1)
fig.add_scatter(x=vht_data.index, y=vht_daily_returns, mode='lines', name='VHT', row=1, col=2)
fig.add_scatter(x=ibb_data.index, y=ibb_daily_returns, mode='lines', name='IBB', row=2, col=1)
fig.add_scatter(x=xbi_data.index, y=xbi_daily_returns, mode='lines', name='XBI', row=2, col=2)
fig.add_scatter(x=ihi_data.index, y=ihi_daily_returns, mode='lines', name='IHI', row=3, col=1)

fig.add_scatter(x=market_index_data.index, y=market_index_daily_returns, mode='lines', name='Market Index', row=3, col=2)

fig.update_layout(title_text="Daily Returns for Healthcare Stocks and Market Index", height=1000, width=1000)
fig.show()


In [67]:
cov_matrix = daily_returns_df.cov()
cov_matrix

,XLV,VHT,IBB,XBI,IHI
XLV,0.000044,0.000044,0.000047,0.000055,0.000045
VHT,0.000044,0.000046,0.000055,0.000072,0.000049
IBB,0.000047,0.000055,0.000106,0.000162,0.000058
XBI,0.000055,0.000072,0.000162,0.000325,0.000085
IHI,0.000045,0.000049,0.000058,0.000085,0.000101


In [68]:
corr_matrix = daily_returns_df.corr()
corr_matrix

,XLV,VHT,IBB,XBI,IHI
XLV,1.000000,0.983161,0.691485,0.459904,0.682623
VHT,0.983161,1.000000,0.780824,0.586943,0.724791
IBB,0.691485,0.780824,1.000000,0.873231,0.563854
XBI,0.459904,0.586943,0.873231,1.000000,0.470927
IHI,0.682623,0.724791,0.563854,0.470927,1.000000


In [69]:
# plot correlation using plotly with cell values and centre aligned title
fig = px.imshow(corr_matrix, color_continuous_scale='RdBu', title="Correlation Matrix between returns of Health Care Sector ETFs")
fig.update_layout(title_x=0.5)
fig.show()

Unlike Technology ETFs, HealthCare Sector ETFs have a large variation in correlation ranging from 0.45 - 0.97.

In [70]:
# standard deviations between daily returns
std_dev = daily_returns_df.std()
std_dev

XLV    0.006632
VHT    0.006796
IBB    0.010294
XBI    0.018034
IHI    0.010032
dtype: float64

In [71]:
# daily mean return
mean_daily_return = daily_returns_df.mean()
mean_daily_return

XLV    0.000575
VHT    0.000549
IBB    0.000291
XBI    0.001002
IHI    0.000426
dtype: float64

## Optimal tangency portfolio using Markowitz model 

In [72]:
# Run the Markowitz Portfolio Optimization
simple_returns_list = list(simple_returns_dict.values())
markowitz = MarkowitzOptimization(simple_returns_list, cov_matrix, risk_free_rate)


In [73]:
# find the weights of the portfolio with short shelling
markowitz_weights_with_ss = markowitz.find_tangency_portfolio_with_short_selling()

In [74]:
# create the wieghts dictionary
markowitz_weights_with_ss_dict = {
    "XLV": markowitz_weights_with_ss[0],
    "VHT": markowitz_weights_with_ss[1],
    "IBB": markowitz_weights_with_ss[2],
    "XBI": markowitz_weights_with_ss[3],
    "IHI": markowitz_weights_with_ss[4]
}

for ticker, weight in markowitz_weights_with_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights with Short Selling in %: ", weight * 100)

XLV -> Markowitz Weights with Short Selling in %:  499.99999999999966
VHT -> Markowitz Weights with Short Selling in %:  -153.04756266301888
IBB -> Markowitz Weights with Short Selling in %:  -363.48928565256193
XBI -> Markowitz Weights with Short Selling in %:  172.54831765817482
IHI -> Markowitz Weights with Short Selling in %:  -56.01146934259364


In [75]:
# find the weights of the portfolio without short shelling
markowitz_weights_without_ss = markowitz.find_tangency_portfolio_no_short_selling()

In [76]:
# weights dictionary
markowitz_weights_without_ss_dict = {
    "XLV": markowitz_weights_without_ss[0],
    "VHT": markowitz_weights_without_ss[1],
    "IBB": markowitz_weights_without_ss[2],
    "XBI": markowitz_weights_without_ss[3],
    "IHI": markowitz_weights_without_ss[4]
}

for ticker, weight in markowitz_weights_without_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling in %: ", weight * 100)

XLV -> Markowitz Weights without Short Selling in %:  88.69467866388241
VHT -> Markowitz Weights without Short Selling in %:  1.4290838477870604e-10
IBB -> Markowitz Weights without Short Selling in %:  2.2364241535066665e-10
XBI -> Markowitz Weights without Short Selling in %:  11.305321335573293
IHI -> Markowitz Weights without Short Selling in %:  1.7774106839119064e-10


In [98]:
expected_return_with_ss, volatility_with_ss = Utils.calculate_portfolio_return_and_risk(markowitz_weights_with_ss, simple_returns_list, cov_matrix)
print("Volatility with Short Selling in %: ", volatility_with_ss * 100)
print("Expected Return with Short Selling in %: ", expected_return_with_ss * 100)

Volatility with Short Selling in %:  1.6117767546115085
Expected Return with Short Selling in %:  64.58343500928866


In [78]:
# calculate sharpe ratio
portfolio_mean_return_with_ss = Utils.calculate_portfolio_mean_return(markowitz_weights_with_ss, mean_daily_return, annualized_factor)
print("Mean Daily Portfolio Return with Short Selling in %: ", portfolio_mean_return_with_ss * 100)

sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(portfolio_mean_return_with_ss, volatility_with_ss, risk_free_rate)
print("Sharpe Ratio with Short Selling: ", sharpe_ratio_with_ss)

Mean Daily Portfolio Return with Short Selling in %:  3.918051149120436
Sharpe Ratio with Short Selling:  0.44325857468244795


In [79]:
expected_returns_without_ss, volatility_without_ss = Utils.calculate_portfolio_return_and_risk(markowitz_weights_without_ss, simple_returns_list, cov_matrix)
print("Volatility without Short Selling in %: ", volatility_without_ss * 100)
print("Expected Return without Short Selling in %: ", expected_returns_without_ss * 100)

Volatility without Short Selling in %:  0.7055778418099218
Expected Return without Short Selling in %:  15.656612081780407


In [80]:
# calculate sharpe ratio
portfolio_mean_return_without_ss = Utils.calculate_portfolio_mean_return(markowitz_weights_without_ss, mean_daily_return, annualized_factor)
print("Mean Daily Portfolio Return without Short Selling in %: ", portfolio_mean_return_without_ss * 100)

sharpe_ratio_without_ss = Utils.calculate_sharpe_ratio(portfolio_mean_return_without_ss, volatility_without_ss, risk_free_rate)
print("Sharpe Ratio without Short Selling: ", sharpe_ratio_without_ss)

Mean Daily Portfolio Return without Short Selling in %:  0.9894367315335816
Sharpe Ratio without Short Selling:  0.05455677224246464


In [99]:
# we are interested in finding optimal weights when an asset can hold 30% of the portfolio max
markowitz_weights_without_ss_pct_constraint = markowitz.find_tangency_portfolio_no_short_selling_and_max_weight(0.3)

In [100]:
markowitz_weights_dict_without_ss_pct_constraint = {
    "XLV": markowitz_weights_without_ss_pct_constraint[0],
    "VHT": markowitz_weights_without_ss_pct_constraint[1],
    "IBB": markowitz_weights_without_ss_pct_constraint[2],
    "XBI": markowitz_weights_without_ss_pct_constraint[3],
    "IHI": markowitz_weights_without_ss_pct_constraint[4]
}

for ticker, weight in markowitz_weights_dict_without_ss_pct_constraint.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling and 30% constraint in %: ", weight * 100)

XLV -> Markowitz Weights without Short Selling and 30% constraint in %:  29.999999999996206
VHT -> Markowitz Weights without Short Selling and 30% constraint in %:  30.00000000000018
IBB -> Markowitz Weights without Short Selling and 30% constraint in %:  4.413136522884997e-12
XBI -> Markowitz Weights without Short Selling and 30% constraint in %:  29.99999999999794
IHI -> Markowitz Weights without Short Selling and 30% constraint in %:  10.000000000001268


In [101]:
expected_returns_without_ss_with_constraint, volatility_without_ss_with_constraint = Utils.calculate_portfolio_return_and_risk(markowitz_weights_without_ss_pct_constraint, simple_returns_list, cov_matrix)
print("Volatility with Short Selling and Constraint in %: ", volatility_without_ss_with_constraint * 100)
print("Expected Return with Short Selling and Constraint in %: ", expected_returns_without_ss_with_constraint * 100)

Volatility with Short Selling and Constraint in %:  0.8945164361272628
Expected Return with Short Selling and Constraint in %:  16.516343705294222


In [102]:
# find the Sharpe Ratio
portfolio_mean_return_with_ss_with_constraint = Utils.calculate_portfolio_mean_return(markowitz_weights_without_ss_pct_constraint, mean_daily_return, annualized_factor)
print("Mean Daily Portfolio Return with Short Selling and Constraint in %: ", portfolio_mean_return_with_ss_with_constraint * 100)

sharpe_no_ss_with_constraint = Utils.calculate_sharpe_ratio(portfolio_mean_return_with_ss_with_constraint, volatility_without_ss_with_constraint, risk_free_rate)
print("Sharpe Ratio with Short Selling and Constraint: ", sharpe_no_ss_with_constraint)

Mean Daily Portfolio Return with Short Selling and Constraint in %:  1.0800841790003597
Sharpe Ratio with Short Selling and Constraint:  0.035665527661760385


## Single Index Market Model

We will now compare results from markowitz model with single index market model. We will use S&P500 Healthcare Index as benchmark to evaluate the performance with Single Index Market Model.

From this point, we will run `SingleFactorModel` class over all ETFs calculating beta, alpha and tau. We will then calculate the expected return using the formula:

$$
u_i = \alpha_i + \beta_i \cdot u_m

In [85]:
def get_simm_params(asset_return: pd.DataFrame, market_return: pd.DataFrame):
  model = SingleFactorModel(asset_return, market_return)
  model.fit()
  
  params = model.get_params()
  alpha = params.get("alpha").get("value")
  beta = params.get("beta").get("value")
  tau = params.get("tau").get("value")
  
  return (alpha, beta, tau)
  

In [86]:
# run SIMM for params
xlv_alpha, xlv_beta, xlv_tau = get_simm_params(xlv_daily_returns, market_index_daily_returns)
vht_alpha, vht_beta, vht_tau = get_simm_params(vht_daily_returns, market_index_daily_returns)
ibb_alpha, ibb_beta, ibb_tau = get_simm_params(ibb_daily_returns, market_index_daily_returns)
xbi_alpha, xbi_beta, xbi_tau = get_simm_params(xbi_daily_returns, market_index_daily_returns)
ihi_alpha, ihi_beta, ihi_tau = get_simm_params(ihi_daily_returns, market_index_daily_returns)


In [87]:
# get expected returns using alpha, beta and expected market return
xlv_expected_return = xlv_alpha + (xlv_beta * market_index_data_expected_return)
vht_expected_return = vht_alpha + (vht_beta * market_index_data_expected_return)
ibb_expected_return = ibb_alpha + (ibb_beta * market_index_data_expected_return)
xbi_expected_return = xbi_alpha + (xbi_beta * market_index_data_expected_return)
ihi_expected_return = ihi_alpha + (ihi_beta * market_index_data_expected_return)

In [88]:
# we can now pack the expected returns into a dictionary along with beta and tau
expected_returns_dict = {
    "XLV": xlv_expected_return,
    "VHT": vht_expected_return,
    "IBB": ibb_expected_return,
    "XBI": xbi_expected_return,
    "IHI": ihi_expected_return
}

alpha_dict = {
    "XLV": xlv_alpha,
    "VHT": vht_alpha,
    "IBB": ibb_alpha,
    "XBI": xbi_alpha,
    "IHI": ihi_alpha
}

beta_dict = {
    "XLV": xlv_beta,
    "VHT": vht_beta,
    "IBB": ibb_beta,
    "XBI": xbi_beta,
    "IHI": ihi_beta
}

tau_dict = {
    "XLV": xlv_tau,
    "VHT": vht_tau,
    "IBB": ibb_tau,
    "XBI": xbi_tau,
    "IHI": ihi_tau
}

for i in range(len(expected_returns_dict.keys())):
  print(f"{ list(expected_returns_dict.keys())[i] } -> Expected Daily Return in %: ", list(expected_returns_dict.values())[i] * 100)
  print(f"{ list(alpha_dict.keys())[i] } -> Alpha: ", list(alpha_dict.values())[i])
  print(f"{ list(beta_dict.keys())[i] } -> Beta: ", list(beta_dict.values())[i])
  print(f"{ list(tau_dict.keys())[i] } -> Tau: ", list(tau_dict.values())[i])
  print()

XLV -> Expected Daily Return in %:  0.057505968538420604
XLV -> Alpha:  6.427122850683221e-05
XLV -> Beta:  0.9994374618383668
XLV -> Tau:  0.0003561933444820635

VHT -> Expected Daily Return in %:  0.054939990349574294
VHT -> Alpha:  3.436629709747204e-05
VHT -> Beta:  1.0077437604736712
VHT -> Tau:  0.0012654165825037814

IBB -> Expected Daily Return in %:  0.029059056034554463
IBB -> Alpha:  -0.00025568686893211737
IBB -> Beta:  1.068877183941388
IBB -> Tau:  0.007486319216384697

XBI -> Expected Daily Return in %:  0.10016452539732099
XBI -> Alpha:  0.0003682023942602926
XBI -> Beta:  1.239429974203785
XBI -> Tau:  0.016088292436845477

IHI -> Expected Daily Return in %:  0.042557626346533046
IHI -> Alpha:  -9.906259704627941e-05
IHI -> Beta:  1.026537941630456
IHI -> Tau:  0.0073897079660244485



In [89]:
# run the SIMM optimization
expected_returns_list = list(expected_returns_dict.values())
beta_list = list(beta_dict.values())
tau_list = list(tau_dict.values())
alpha_list = list(alpha_dict.values())

simm_optimization = SingleFactorModelOptimization(expected_returns_list, beta_list, tau_list, market_index_data_variance, risk_free_rate)

In [90]:
# find the optimal weights when short selling is allowed
simm_weights_with_ss = simm_optimization.find_weights_short_selling_allowed()

In [91]:
simm_weights_with_ss_dict = {
  "XLV": simm_weights_with_ss[0],
  "VHT": simm_weights_with_ss[1],
  "IBB": simm_weights_with_ss[2],
  "XBI": simm_weights_with_ss[3],
  "IHI": simm_weights_with_ss[4]
}

for ticker, weight in simm_weights_with_ss_dict.items():
    print(f"{ticker} -> SIMM Weights with Short Selling in %: ", weight * 100)

XLV -> SIMM Weights with Short Selling in %:  121.80507438292378
VHT -> SIMM Weights with Short Selling in %:  -11.429765455801668
IBB -> SIMM Weights with Short Selling in %:  -4.6542351825081125
XBI -> SIMM Weights with Short Selling in %:  -4.088766827807678
IHI -> SIMM Weights with Short Selling in %:  -1.6323069168063136


With weights found, we can find the expected return of portfolio and the risk using the formula:

$$
u_p = \alpha_p + \beta_p \cdot u_m

\newline

\sigma_p^2 = \beta_p^2 \cdot \sigma_m^2 + \tau_p^2

\newline

where:
\newline

\alpha_p = \sum_{i=1}^{n} x_i \cdot \alpha_i

\newline

\beta_p = \sum_{i=1}^{n} x_i \cdot \beta_i

\newline

\tau_p^2 = \sum_{i=1}^{n} x_i^2 \cdot \tau_i^2

In [92]:
simm_expected_returns_with_ss, simm_portfolio_risk_with_ss = Utils.calculate_factor_model_percentage_portfolio_return_and_risk(simm_weights_with_ss, market_index_data_expected_return, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Portfolio Risk with Short Selling in %: ", simm_portfolio_risk_with_ss * 100)
print("Portfolio Expected Return with Short Selling in %: ", simm_expected_returns_with_ss * 100)

Portfolio Risk with Short Selling in %:  0.8164270406366649
Portfolio Expected Return with Short Selling in %:  98.50011233851194


In [93]:
# find the sharpe ratio
simm_portfolio_mean_return_with_ss = Utils.calculate_portfolio_mean_return(simm_weights_with_ss, mean_daily_return, annualized_factor)
print("Mean Daily Portfolio Return with Short Selling in %: ", simm_portfolio_mean_return_with_ss * 100)

simm_sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(simm_portfolio_mean_return_with_ss, simm_portfolio_risk_with_ss, risk_free_rate)
print("SIMM Sharpe Ratio with Short Selling: ", simm_sharpe_ratio_with_ss)

Mean Daily Portfolio Return with Short Selling in %:  0.9147373055437857
SIMM Sharpe Ratio with Short Selling:  0.018894919010130625


In [94]:
# find optimal weights when short selling is not allowed
simm_weights_no_ss = simm_optimization.find_weights_no_short_selling()

In [95]:
simm_weights_no_ss_dict = {
  "XLV": simm_weights_no_ss[0],
  "VHT": simm_weights_no_ss[1],
  "IBB": simm_weights_no_ss[2],
  "XBI": simm_weights_no_ss[3],
  "IHI": simm_weights_no_ss[4]
}

for ticker, weight in simm_weights_no_ss_dict.items():
  print(f"{ticker} -> SIMM Weights without Short Selling in %: ", weight * 100)
    

XLV -> SIMM Weights without Short Selling in %:  92.13986399772214
VHT -> SIMM Weights without Short Selling in %:  7.361168629210132
IBB -> SIMM Weights without Short Selling in %:  0.22307752044484755
XBI -> SIMM Weights without Short Selling in %:  0.056010501041277284
IHI -> SIMM Weights without Short Selling in %:  0.21987935158159466


In [96]:
# we will now work to find the expected return and risk of the portfolio
simm_expected_returns_without_ss, simm_portfolio_risk_no_ss = Utils.calculate_factor_model_percentage_portfolio_return_and_risk(simm_weights_no_ss, market_index_data_expected_return, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Portfolio Risk without Short Selling in %: ", simm_portfolio_risk_no_ss * 100)
print("Portfolio Expected Return without Short Selling in %: ", simm_expected_returns_without_ss * 100)

Portfolio Risk without Short Selling in %:  0.6130953845503232
Portfolio Expected Return without Short Selling in %:  100.03978475270026


In [97]:
# find the sharpe ratio
simm_portfolio_mean_return_without_ss = Utils.calculate_portfolio_mean_return(simm_weights_no_ss, mean_daily_return, annualized_factor)
print("Mean Daily Portfolio Return without Short Selling in %: ", simm_portfolio_mean_return_without_ss * 100)

simm_sharpe_ratio_no_ss = Utils.calculate_sharpe_ratio(simm_portfolio_mean_return_without_ss, simm_portfolio_risk_no_ss, risk_free_rate)
print("SIMM Sharpe Ratio without Short Selling: ", simm_sharpe_ratio_no_ss)

Mean Daily Portfolio Return without Short Selling in %:  0.9087306283738461
SIMM Sharpe Ratio without Short Selling:  0.056820149898518704
